In [48]:
#module import
import urllib.request
from bs4 import BeautifulSoup
import re
import pandas as pd
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, "english_uk") #as we need to deal with names of monthes later on.
import os
import csv
import psycopg2

In [50]:
# Assigning the IMO vessel identifiers to the IMOS variable as a list

IMOS = [9545663 , 9424508, 9811139, 9660114, 8848288]


In [59]:
#Making use of BeautifulSoup to scrape data from vesselfinder

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
items = []
for IMO in IMOS:
    url = r'https://www.vesselfinder.com/en/vessels/VOS-TRAVELLER-IMO-' + str(IMO)
    req = urllib.request.Request(url, None, hdr)
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
    parsed_html = BeautifulSoup(the_page)
    tables = parsed_html.findAll("table")
    for table in tables:
        if table.findParent("table") is None:
            for row in table.findAll('tr'):
                aux = row.findAll('td')
                try:
                    if aux[0].string == "Coordinates":
                        coords = aux[1].string
                    if aux[0].string == "Vessel Name":
                        name = aux[1].string
                    if aux[0].string == "Last report":
                        time = datetime.strptime(aux[1].contents[1], ' %b %d, %Y %I:%M %Z')
                except: 
                    print("strange table found")
    coordsSplit = coords.split("/")
    def dms2dd(degrees,direction):
        dd = float(degrees) ;
        if direction == 'S' or direction == 'W':
            dd *= -1
        return dd
    def parse_dms(dms):
        parts = re.split(' ', dms)
        lat = dms2dd(parts[0], parts[1])
        return lat
    lat = parse_dms(coordsSplit[0])
    lng = parse_dms(coordsSplit[1])

    items.append((lat, lng, name, time))

strange table found
strange table found
strange table found
strange table found
strange table found
strange table found
strange table found
strange table found
strange table found


In [60]:
#Import the tuple into a PostgreSQL table 

import psycopg2
conn = psycopg2.connect("host=localhost dbname=nsea user=postgres")
cur = conn.cursor()
for d in items:
    cur.execute("INSERT into vesseltracking(lat,lon,name,time ) VALUES (%s, %s, %s,%s)", d)
    cur.execute("UPDATE vesseltracking SET geom =(ST_SetSRID(ST_MakePoint(lon,lat),4326));")
    cur.execute("NOTIFY qgis, 'refresh qgis';")
    cur.execute("""DELETE FROM
    vesseltracking a
        USING vesseltracking b
WHERE
    a.id < b.id
    AND a.lat = b.lat
    AND b.lon = b.lon;""")

conn.commit()